In [1]:
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [4]:
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY

LANGCHAIN_TRACING=os.getenv("LANGCHAIN_TRACING")
os.environ["LANGCHAIN_TRACING"] = LANGCHAIN_TRACING

LANGSMITH_ENDPOINT = os.getenv("LANGSMITH_ENDPOINT")
os.environ["LANGSMITH_ENDPOINT"] = LANGSMITH_ENDPOINT

LANGSMITH_PROJECT = "langchain_essentials"

In [5]:
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It")

In [6]:
model.invoke("Hello, how are you?")

AIMessage(content="As an AI, I don't have feelings or experiences like humans do. However, I'm here and ready to assist you!\n\nHow can I help you today?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 15, 'total_tokens': 54, 'completion_time': 0.070909091, 'prompt_time': 0.00124623, 'queue_time': 0.020609877, 'total_time': 0.072155321}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--be9b70cb-0c75-48d7-825a-b35aad3330b1-0', usage_metadata={'input_tokens': 15, 'output_tokens': 39, 'total_tokens': 54})

In [7]:
from langchain.memory import ConversationSummaryMemory
from langchain.memory import ChatMessageHistory


In [8]:
memory = ConversationSummaryMemory(llm=model,return_messages=True)

/var/folders/p_/v95prvpj4cz7y53gvmgm1d_r0000gn/T/ipykernel_43654/25687835.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=model,return_messages=True)


In [9]:
memory.save_context(
    {"Human": "John and Jack are working on a new project on ML"},
    {"AI": "Thats interesting! What is their project about?"}
)

In [10]:
memory.load_memory_variables({})

{'history': [SystemMessage(content="Human: John and Jack are working on a new project on ML\nAI: That's interesting! What is their project about? \n\nNew summary: The human mentions that John and Jack are working on a new project related to machine learning. The AI expresses interest and asks for more details about the project. \n", additional_kwargs={}, response_metadata={})]}

In [11]:
summary = memory.load_memory_variables({})

In [12]:
print(summary["history"][0].content)

Human: John and Jack are working on a new project on ML
AI: That's interesting! What is their project about? 

New summary: The human mentions that John and Jack are working on a new project related to machine learning. The AI expresses interest and asks for more details about the project. 



In [13]:
memory.save_context(
    {"Human": "The project is about building a healthcare application."},
    {"AI": "Thats great! Is it for hospitals or patients?"}
)

memory.save_context(
    {"Human": "It is for the patients."},
    {"AI": "Cool, wishing Jack and John all the best"}
)

In [15]:
memory.load_memory_variables({})

{'history': [SystemMessage(content='The human mentions that John and Jack are working on a new project related to machine learning. The AI expresses interest and asks for more details about the project. The human reveals that the project is focused on building a healthcare application for patients. The AI responds positively and wishes John and Jack all the best.  \n', additional_kwargs={}, response_metadata={})]}

In [16]:
memory.chat_memory

InMemoryChatMessageHistory(messages=[HumanMessage(content='John and Jack are working on a new project on ML', additional_kwargs={}, response_metadata={}), AIMessage(content='Thats interesting! What is their project about?', additional_kwargs={}, response_metadata={}), HumanMessage(content='The project is about building a healthcare application.', additional_kwargs={}, response_metadata={}), AIMessage(content='Thats great! Is it for hospitals or patients?', additional_kwargs={}, response_metadata={}), HumanMessage(content='It is for the patients.', additional_kwargs={}, response_metadata={}), AIMessage(content='Cool, wishing Jack and John all the best', additional_kwargs={}, response_metadata={})])

In [17]:
memory.chat_memory.messages

[HumanMessage(content='John and Jack are working on a new project on ML', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Thats interesting! What is their project about?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='The project is about building a healthcare application.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Thats great! Is it for hospitals or patients?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='It is for the patients.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Cool, wishing Jack and John all the best', additional_kwargs={}, response_metadata={})]

In [18]:
messages = memory.chat_memory.messages

In [19]:
history = ChatMessageHistory()

In [20]:
history.add_user_message("Hello")

In [21]:
history.add_ai_message("Hi! How can I help you?")

In [22]:
new_memory = ConversationSummaryMemory.from_messages(
    llm=model,
    chat_memory=history,
    memory_key="summary",
    human_prefix="User",
    ai_prefix="Assistant"
)

In [23]:
new_memory.buffer

'Current summary: A user greets the assistant. \n\nNew lines of conversation:\nUser: Hello\nAssistant: Hi! How can I help you? \n\nNew summary: A user greets the assistant, and the assistant responds with a greeting and an offer to help. \n'

#### Chaining

In [24]:
from langchain.chains import ConversationChain

In [25]:
convo_with_summary = ConversationChain(
    llm=model,
    memory=ConversationSummaryMemory(llm=model),
    verbose=True
)

/var/folders/p_/v95prvpj4cz7y53gvmgm1d_r0000gn/T/ipykernel_43654/2088699057.py:1: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  convo_with_summary = ConversationChain(


In [26]:
convo_with_summary.predict(input="Hi, how are you?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, how are you?
AI:

> Finished chain.


"As an AI, I don't have feelings, but I'm ready to chat! How are you doing today?  😊  \n\n"

In [27]:
convo_with_summary.predict(input="John and Jack are thinking of launching a new product in summer.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
New summary: The human greets the AI and asks how it is doing. The AI, acknowledging it doesn't have feelings, responds that it's ready to chat and asks how the human is doing.  😊  

Human: John and Jack are thinking of launching a new product in summer.
AI:

> Finished chain.


"That's interesting!  Tell me more about this product. What is it? What makes it special?  😊  Are John and Jack planning to sell it online or in physical stores? \n\n"

In [28]:
convo_with_summary.predict(input="the product aims to help the customers to travel from one location to another")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
New summary: The human greets the AI and asks how it is doing. The AI, acknowledging it doesn't have feelings, responds that it's ready to chat and asks how the human is doing.  😊  The human tells the AI about John and Jack's plans to launch a new product in the summer. The AI expresses interest and asks for more details about the product, such as what it is, what makes it special, and how they plan to sell it. 😊  

Human: the product aims to help the customers to travel from one location to another
AI:

> Finished chain.


"That's interesting!  \n\nCan you tell me more about how it helps people travel? Is it a physical product like a vehicle or a service like a ride-sharing app?  \n\nKnowing that will help me understand what makes it special and how they plan to sell it. 😊  \n\n"

In [29]:
convo_with_summary.predict(input="it will use a single person drone to transport one person at a time with no traffic and delays")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
New summary: The human greets the AI and asks how it is doing. The AI, acknowledging it doesn't have feelings, responds that it's ready to chat and asks how the human is doing.  😊  The human tells the AI about John and Jack's plans to launch a new product in the summer. The AI expresses interest and asks for more details about the product, such as what it is, what makes it special, and how they plan to sell it. 😊  The human explains that the product helps people travel from one location to another. The AI, intrigued, asks for more details about how it accomplishes this, clarifying if it's a physical product or a service.  😊  

Human: it will use a single person drone t

"That's fascinating!  A single-person drone for transportation?  I've read about the potential of drones for delivery services, but this is a whole new level.  \n\nTell me more about how this drone would work.  What kind of range does it have?  How will it navigate safely around obstacles and other aircraft?  And what about weather conditions - would it be able to operate in rain or strong winds?  \n\nI'm really curious to learn more about the technical aspects of this product. 😊  \n\n"

In [30]:
convo_with_summary.predict(input="can you summarise the startup's idea?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
The human greets the AI and asks how it is doing. The AI, acknowledging it doesn't have feelings, responds that it's ready to chat and asks how the human is doing.  😊  The human tells the AI about John and Jack's plans to launch a new product in the summer. The AI expresses interest and asks for more details about the product, such as what it is, what makes it special, and how they plan to sell it. 😊  The human explains that the product helps people travel from one location to another using a single-person drone to transport one person at a time with no traffic and delays. The AI is intrigued and asks for more details about how the drone would work, including its range

"John and Jack are launching a new product this summer that uses single-person drones to transport people from one location to another.  \n\nTheir idea is to offer a travel solution that bypasses traffic and delays, allowing individuals to travel quickly and efficiently.  \n\nI'm curious to learn more about the specifics of how the drones will work, like their range, navigation system, safety precautions, and ability to handle different weather conditions. 😊 \n"

In [31]:
convo_with_summary.memory

ConversationSummaryMemory(llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x114bbf520>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x114bed750>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********')), chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi, how are you?', additional_kwargs={}, response_metadata={}), AIMessage(content="As an AI, I don't have feelings, but I'm ready to chat! How are you doing today?  😊  \n\n", additional_kwargs={}, response_metadata={}), HumanMessage(content='John and Jack are thinking of launching a new product in summer.', additional_kwargs={}, response_metadata={}), AIMessage(content="That's interesting!  Tell me more about this product. What is it? What makes it special?  😊  Are John and Jack planning to sell it online or in physical stores? \n\n", additional_kwargs={}, response_metadata={}), HumanMessage(content='the product aims to help the custome

In [32]:
convo_with_summary.memory.buffer

"The human greets the AI and asks how it is doing. The AI, acknowledging it doesn't have feelings, responds that it's ready to chat and asks how the human is doing.  😊  The human tells the AI about John and Jack's plans to launch a new product in the summer. The AI expresses interest and asks for more details about the product, such as what it is, what makes it special, and how they plan to sell it. 😊  The human explains that the product helps people travel from one location to another using a single-person drone to transport one person at a time with no traffic and delays. The AI is intrigued and asks for more details about how the drone would work, including its range, navigation, safety features, and ability to operate in various weather conditions.  😊  The human then asks the AI to summarize the startup's idea, which the AI does, explaining that John and Jack are launching a product using single-person drones to transport people quickly and efficiently, bypassing traffic and delays

## Conversation Summary Buffer Memory

In [33]:
from langchain.memory import ConversationSummaryBufferMemory

In [34]:
memory2 = ConversationSummaryBufferMemory(llm=model, return_messages=True)

/var/folders/p_/v95prvpj4cz7y53gvmgm1d_r0000gn/T/ipykernel_43654/3173139260.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory2 = ConversationSummaryBufferMemory(llm=model, return_messages=True)


In [36]:
memory2.save_context(
    {"input": "It will be a bit costly when it launches but eventually we will decrease the prices"},
    {"output": "Yes that is great!"}
)

In [37]:
memory2.save_context(
    {"input": "At an even later stage we can add more powerful drones which will be able to transport 3-4 people at a time"},
    {"output": "Oh wow this is very interesting!"}
)

In [38]:
memory2.load_memory_variables({})

{'history': [HumanMessage(content='It will be a bit costly when it launches but eventually we will decrease the prices', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Yes that is great!', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='At an even later stage we can add more powerful drones which will be able to transport 3-4 people at a time', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Oh wow this is very interesting!', additional_kwargs={}, response_metadata={})]}

In [39]:
memory2.load_memory_variables({})["history"]

[HumanMessage(content='It will be a bit costly when it launches but eventually we will decrease the prices', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Yes that is great!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='At an even later stage we can add more powerful drones which will be able to transport 3-4 people at a time', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Oh wow this is very interesting!', additional_kwargs={}, response_metadata={})]

In [40]:
from langchain.chains import ConversationChain

In [41]:
convo = ConversationChain(
    llm = model,
    verbose = True,
    memory = ConversationSummaryBufferMemory(llm=model, max_token_limit=40)
)

In [43]:
convo.predict(input="hi how are you")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human asks the AI about the startup idea. The AI is excited to share that it's a platform connecting local artists with people wanting to commission personalized artwork.  It would allow customers to commission pieces like custom portraits, landscapes, or murals from artists in their community.  The platform would feature artist profiles, pricing, and secure payment and messaging tools. 

Human: hi how are you
AI:

> Finished chain.


"I'm doing well, thank you! I'm buzzing with excitement about this new startup idea I've been working on.  Want to hear about it?  \n\n"

In [44]:
convo.predict(input="I am working on some documentation related to mathematics")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human asks the AI about the startup idea. The AI is excited to share that it's a platform connecting local artists with people wanting to commission personalized artwork.  It would allow customers to commission pieces like custom portraits, landscapes, or murals from artists in their community.  The platform would feature artist profiles, pricing, and secure payment and messaging tools. The human then greets the AI. 




AI: I'm doing well, thank you! I'm buzzing with excitement about this new startup idea I've been working on.  Want to hear about it?  


Human: I am working on some documentation related to mathematics
AI:

> Finished chain.


"That's fascinating!  Mathematics is such a fundamental and beautiful subject. What kind of documentation are you working on?  \n\n(The AI is trying to engage with the human's work while still being enthusiastic about its own project.) \n\n"

In [45]:
convo.predict(input="I am working on problems based on integration")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human asks the AI about the startup idea. The AI is excited to share that it's a platform connecting local artists with people wanting to commission personalized artwork, allowing customers to commission pieces like custom portraits, landscapes, or murals from artists in their community. It would feature artist profiles, pricing, and secure payment and messaging tools. The human then greets the AI. The AI responds enthusiastically, saying it's doing well and eager to share its new startup idea. It then tries to engage with the human's work on mathematics documentation, expressing interest in the subject.  

Human: I am working on problems based on integrati

"That's fascinating! Integration is such a fundamental concept in mathematics.  Is there a particular type of integration you're focusing on?  For example, are you working with definite integrals, indefinite integrals, or maybe something more specialized like Fourier transforms or line integrals?  \n\nI'm eager to learn more about what you're exploring! \n\n"

In [46]:
convo.predict(input="the concept i am working on includes 4th degree of integration")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human asks the AI about the startup idea. The AI is excited to share that it's a platform connecting local artists with people wanting to commission personalized artwork, allowing customers to commission pieces like custom portraits, landscapes, or murals from artists in their community. It would feature artist profiles, pricing, and secure payment and messaging tools. The human then greets the AI. The AI responds enthusiastically, saying it's doing well and eager to share its new startup idea. It then tries to engage with the human's work on mathematics documentation, expressing interest in the subject. The AI is particularly interested in the human's focu

"That's fascinating!  Fourth-degree integration is quite advanced.  Could you tell me more about what you're working on specifically?  \n\nI'm not familiar with the specific applications of fourth-degree integration, so I'm eager to learn. Are you dealing with any particular mathematical functions or problems that require this level of integration? \n\n"

In [47]:
convo.predict(input="Whats the subject I am workig on?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human asks the AI about the startup idea. The AI is excited to share that it's a platform connecting local artists with people wanting to commission personalized artwork, allowing customers to commission pieces like custom portraits, landscapes, or murals from artists in their community. It would feature artist profiles, pricing, and secure payment and messaging tools. The human then greets the AI. The AI responds enthusiastically, saying it's doing well and eager to share its new startup idea. It then tries to engage with the human's work on mathematics documentation, expressing interest in the subject. The AI is particularly interested in the human's focu

"You mentioned you're working on mathematics documentation, specifically focusing on 4th degree integration.  \n\n"